# 2. Preprocesado de los datos - Detección de *Fake News*

José Luis Aguilera Luzania

## Introducción

### ¿Qué es la detección de *Fake News*?
- Es la tarea de evaluar la veracidad de las afirmaciones en las noticias.
- Es un problema crítico en Procesamiento del Lenguaje Natural porque tanto en medios de noticias tradicionales como en medios digitales las Fake News generan un gran impacto social y político en cada individuo.
- La exposición a las Fake News puede generar actitudes de ineficacia, alienación y cinismo hacia ciertos candidatos políticos (Balmas, 2014).

### Objetivo de la libreta
El objetivo de esta libreta es separar los datos que necesitamos de todo el conjunto de datos, limpiar los datos necesarios y estructurarlos para un análisis exploratorio.

## Preprocesamiento de los datos

### Librerías

- Manipulación de datos:
    - `pandas`: Librería para manipular los datos de forma tabular.
    - `matplotlib`: Librería para graficar.
    - `openpyxl`: Librería para leer archivos de hoja de cálculo en pandas.
    - `cmd`: Librería para controlar el formato de impresión en la consola.
    - `unicode`: Librería para eliminar acentos de las palabras.
    - `re`: Librería para utilizar expresiones regulares.
    
- Procesamiento del lenguaje natural:
    - `nltk`: Librería para utilizar técnicas de procesameinto del lenguaje natural.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import cmd
import unidecode
import re
import nltk

### Datos adicionales para la librería `nltk`
- `punkt`: Necesario para utilizar el tokenizador de los textos.
- `stopwords`: Palabras comunes que no añaden información, como: el, la, los, etc.

In [3]:
# Descargar los datos necesarios.
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JoseLuis_AL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JoseLuis_AL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Leer los datos

Los datos estan divididos en los archivos `train.xlsx` y `development.xlsx`, con 80% para entrenamiento y 20% para pruebas respectivamente.

Para cargar los conjuntos de datos se utilizará la librería `pandas` y su estructura `DataFrame`.

In [4]:
df_entrenamiento = pd.read_excel('Datos/train.xlsx')
df_pruebas = pd.read_excel('Datos/development.xlsx')

### Conjunto de entrenamiento

In [5]:
print(f'Noticias, Columnas: {df_entrenamiento.shape}')
print(f'Columnas: {list(df_entrenamiento.columns)}')
print('Noticias verdaderas: {}'.format(df_entrenamiento.groupby('Category').size()[0]))
print('Noticias falsas: {}'    .format(df_entrenamiento.groupby('Category').size()[1]))

Noticias, Columnas: (676, 7)
Columnas: ['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']
Noticias verdaderas: 338
Noticias falsas: 338


### Conjunto de pruebas

In [6]:
print(f'Noticias, Columnas: {df_pruebas.shape}')
print(f'Columnas: {list(df_pruebas.columns)}')
print('Noticias verdaderas: {}'.format(df_pruebas.groupby('Category').size()[0]))
print('Noticias falsas: {}'    .format(df_pruebas.groupby('Category').size()[1]))

Noticias, Columnas: (295, 7)
Columnas: ['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']
Noticias verdaderas: 142
Noticias falsas: 153


### ¿Porqué limpiar los datos?

Como se observo durante el análisis de los datos en la libreta 1, varias fuentes se econtraban escritas de manera incorrecta de tal forma que el número de fuentes encontradas en un inicio era mayor al número de fuentes real.  

En esta libreta se impedirán problemas futuros al extraer los datos necesarios y limpiarlos.  

El proceso para obtener los datos que necesitamos y como queremos es el siguiente:

1. Eliminar las columna `Link`.
2. Procesar las fuentes (`Source`).
3. Procesar los encabezados (`Headline`).
4. Procesar el texto (`Text`).

### Eliminar las columnas `Id` y `Link`

La columna `Link` es útil para leer el articulo de donde se obtuvo la noticia, pero al momento de realizar un análisis exploratorio o procesar el texto para utilizarlo en un algoritmo de clasificación no es muy útil.

In [7]:
# Columnas a eliminar.
columnas = ['Id', 'Link']

# Actualizar los dataframe.
df_entrenamiento = df_entrenamiento.drop(columns=['Id', 'Link'])
df_pruebas = df_pruebas.drop(columns=['Id', 'Link'])

In [8]:
# Comprobar las columnas.
print(f'Columnas entrenamiento: {list(df_entrenamiento.columns)}')
print(f'Columnas pruebas: {list(df_pruebas.columns)}')

Columnas entrenamiento: ['Category', 'Topic', 'Source', 'Headline', 'Text']
Columnas pruebas: ['Category', 'Topic', 'Source', 'Headline', 'Text']


### Procesar textos

Los textos de las columnas deben ser procesados para evitar problemas como el de las fuentes duplicadas por tener faltas de ortografía. 

Para procesar los textos se utilizará la misma función que se utilizo en la libreta 1 que ahora se llamará `procesar_textos` y consta de las siguientes reglas:

1. Convertir todas las palabras en minúsculas.
2. Eliminar acentos.
3. Eliminar todo excepto las palabras.

También se usará una función auxiliar llamada `procesar_textos_df` para procesar solo los datos de las columnas especificadas.

In [9]:
def procesar_texto(texto):
    '''
    Input: 
        texto: una cadena de texto.
        
    Output:
        texto: una cadena de texto formada solo por las palabras del texto original, sin acentos, caracteres especiales o espacios extra.
    '''
    # 1. Convertir el texto en minúsculas.
    texto = texto.lower()
    
    # 2. Eliminar acentos.
    texto = unidecode.unidecode(texto)
    
    # 3. Quedarnos solo con las palabras.
    texto = re.findall(r'\w+', texto)
    texto = ' '.join(texto)
    
    return texto

In [10]:
def procesar_textos_df(df, columnas, funcion_procesar):
    
    for col in columnas:
        df[col] = df[col].apply(funcion_procesar)
    
    return df

Aplicamos la función a los conjuntos de datos.

In [11]:
# Lista de columnas a procesar.
columnas_a_procesar = ['Category', 'Topic', 'Source', 'Headline', 'Text']

# Procesamiento.
df_entrenamiento = procesar_textos_df(df_entrenamiento, columnas_a_procesar, procesar_texto)
df_pruebas = procesar_textos_df(df_pruebas, columnas_a_procesar, procesar_texto)

### Renombrar las columnas de los DataFrame.

Las columnas se encuentran escritas en inglés pero los datos como el texto en español, para evitar inconsistencias en el análisis exploratorio las columnas se traducirán al español.

In [12]:
columnas_dict = {
    'Category': 'Categoria',
    'Topic': 'Tema',
    'Source': 'Fuente',
    'Headline': 'Encabezado',
    'Text': 'Texto'
}

df_entrenamiento.rename(columns=columnas_dict, inplace=True)
df_pruebas.rename(columns=columnas_dict, inplace=True)

### Guardar los DataFrame en archivos de texto.

Para utilizar los conjuntos de datos `df_entrenamiento` y `df_pruebas`, se guardarán en archivos de extensión `.csv` llamados `datos_entrenamiento.csv` y `datos_pruebas.csv`

In [13]:
df_entrenamiento.to_csv('Datos/datos_entrenamiento.csv', index=False)
df_pruebas.to_csv('Datos/datos_pruebas.csv', index=False)